In [113]:
def IntegersModP(p):
    class IntegerModP():
        def __init__(self, n):
            if (n is None):
                sl
            self.n = n % p
            self.field = IntegerModP
        def __add__(self, other): return IntegerModP(self.n + other.n)
        def __sub__(self, other): return IntegerModP(self.n - other.n)
        def __mul__(self, other): return IntegerModP(self.n * other.n)
        def __truediv__(self, other): return self * other.inverse()
        def __div__(self, other): return self * other.inverse()
        def __neg__(self): return IntegerModP(-self.n)
        def __eq__(self, other): return isinstance(other, IntegerModP) and self.n == other.n
        def __abs__(self): return abs(self.n)
        def __str__(self): return '%d (mod %d)' % (self.n, self.p)
        def __repr__(self): return '%d (mod %d)' % (self.n, self.p)
        def __pow__(self,m): return IntegerModP(self.n**m)
        def inverse(self):
            return IntegerModP(self.n**(self.p-2))
    IntegerModP.p = p
    IntegerModP.__name__ = 'F_%d' % (p)
    return IntegerModP

def MyEllipticCurve(p,myA=None,myB=None,myJ=None):
    class EllipticCurve():
        # Assumes x,y given as either int or IntegerModP. So if it is not an integer, assumes it is IntegerModP with correct p
        #     this may cause a problem
        def __init__(self,x,y,isPInf):
            self.whichCurve=EllipticCurve
            if (type(x)==int):
                self.x=self.whichCurve.groundField(x)
            else:
                self.x=x
            if (type(y)==int):
                self.y=self.whichCurve.groundField(y)
            else:
                self.y=y
            self.isPInf=isPInf
        # double and add are assuming that the curve is given with A,B coefficients rather than Legendre form
        #   does not check this. Potential problem.
        def double(self):
            if (self.isPInf):
                return self
            myGF = self.whichCurve.groundField
            s = myGF(3)*self.x**2+myGF(self.whichCurve.myA)
            s = s/(myGF(2)*self.y)
            xR = s**2-myGF(2)*self.x
            return self.whichCurve(xR,-self.y-s*(xR-self.x),False)
        def ecAdd(self,other):
            if (self.isPInf):
                return other
            elif (other.isPInf):
                return self
            elif (self.x==other.x):
                if (self.y==-other.y):
                    # put two junk numbers for x,y because it will be PInf anyway
                    return self.whichCurve(2894,19204,True)
                else:
                    return self.double()
            else:
                s = (self.y - other.y)/(self.x-other.x)
                xR = s**2 - self.x - other.x
                return self.whichCurve(xR,-self.y-s*(xR-self.x),False)
        def __str__(self):
            return '(%s,%s, %r)' % (str(self.x),str(self.y),self.isPInf)
        def jacobiEmbedding(self):
            if(self.whichCurve.myJ is None):
                return None
            else:
                if self.isPInf:
                    return (self.whichCurve.groundField(0),self.whichCurve.groundField(1),self.whichCurve.groundField(1),self.whichCurve.groundField(1))
                else:
                    x2 = (self.x)**2
                    x3 = self.whichCurve.groundField(2)*(self.x)
                    jGF = self.whichCurve.groundField(self.whichCurve.myJ)
                    return (-self.whichCurve.groundField(2)*(self.y),
                            x2-jGF,x2+x3*jGF+jGF,
                            x2+x3+jGF)
    EllipticCurve.groundField=IntegersModP(p)
    EllipticCurve.myA=myA
    EllipticCurve.myB=myB
    EllipticCurve.myJ=myJ
    EllipticCurve.__name__= 'myA=%s, myB=%s, myJ=%s over %s' % (str(myA),str(myB),str(myJ),IntegersModP(p).__name__)
    return EllipticCurve

In [114]:
myF19=IntegersModP(19)
print(myF19.__name__)
print(myF19(9)*myF19(10))
result=pow(myF19(9),2)
print(result)
result=myF19(9)**2
print(result)
type(result)

F_19
14 (mod 19)
5 (mod 19)
5 (mod 19)


__main__.IntegersModP.<locals>.IntegerModP

In [117]:
myCurve=MyEllipticCurve(19,myJ=4)
print(myCurve.__name__)
point1=myCurve(4,0,False)
result=point1.jacobiEmbedding()
print(result)
point1=myCurve(4,0,True)
result=point1.jacobiEmbedding()
print(result)
myCurve=MyEllipticCurve(7,myA=4,myB=16)
point1=myCurve(0,4,False)
print('P')
print(point1)
point2=point1.ecAdd(point1)
print('2P')
print(point2)
for i in range(7):
    point2=point2.ecAdd(point1)
    print('%d P' % (i+3))
    print(point2)

myA=None, myB=None, myJ=4 over F_19
(0 (mod 19), 12 (mod 19), 14 (mod 19), 9 (mod 19))
(0 (mod 19), 1 (mod 19), 1 (mod 19), 1 (mod 19))
P
(0 (mod 7),4 (mod 7), False)
doubling
2P
(2 (mod 7),2 (mod 7), False)
3 P
(6 (mod 7),2 (mod 7), False)
4 P
(5 (mod 7),0 (mod 7), False)
5 P
(6 (mod 7),5 (mod 7), False)
6 P
(2 (mod 7),5 (mod 7), False)
7 P
(0 (mod 7),3 (mod 7), False)
8 P
(3 (mod 7),3 (mod 7), True)
9 P
(0 (mod 7),4 (mod 7), False)
